## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-06-22-50-15 +0000,nypost,Newark International Airport facing flight del...,https://nypost.com/2025/11/06/us-news/newark-i...
1,2025-11-06-22-50-00 +0000,wsj,Opinion | Take Mamdani Seriously and Literally,https://www.wsj.com/opinion/take-mamdani-serio...
2,2025-11-06-22-50-00 +0000,wsj,Virginia voters elected Jay Jones as attorney ...,https://www.wsj.com/politics/elections/jay-jon...
3,2025-11-06-22-48-10 +0000,bbc,Elon Musk's $1tn pay deal approved by Tesla sh...,https://www.bbc.com/news/articles/cwyk6kvyxvzo...
4,2025-11-06-22-46-00 +0000,wsj,Opinion | The FDA’s Meltdown Escalates,https://www.wsj.com/opinion/fda-pharma-george-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
56,trump,47
75,new,18
11,will,16
14,mamdani,15
10,shutdown,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
91,2025-11-06-20-57-06 +0000,latimes,Supreme Court rules Trump may remove transgend...,https://www.latimes.com/politics/story/2025-11...,103
367,2025-11-05-23-44-26 +0000,nyt,Trump vs. Mamdani: President Sees a Capable Fo...,https://www.nytimes.com/2025/11/05/us/politics...,100
28,2025-11-06-22-27-00 +0000,wsj,Trump Administration Must Fully Fund SNAP Bene...,https://www.wsj.com/politics/policy/trump-admi...,98
172,2025-11-06-17-38-00 +0000,wsj,Many GOP lawmakers are saying no—for now—to Pr...,https://www.wsj.com/politics/policy/senate-rep...,98
135,2025-11-06-19-23-00 +0000,wsj,Former House Speaker Nancy Pelosi says she won...,https://www.wsj.com/politics/nancy-pelosi-reti...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
91,103,2025-11-06-20-57-06 +0000,latimes,Supreme Court rules Trump may remove transgend...,https://www.latimes.com/politics/story/2025-11...
135,94,2025-11-06-19-23-00 +0000,wsj,Former House Speaker Nancy Pelosi says she won...,https://www.wsj.com/politics/nancy-pelosi-reti...
92,69,2025-11-06-20-54-00 +0000,wsj,Employers said they cut more than a million jo...,https://www.wsj.com/economy/jobs/planned-u-s-j...
184,68,2025-11-06-17-02-00 +0000,wsj,The FAA said it was ordering airline traffic t...,https://www.wsj.com/business/airlines/u-s-to-r...
164,56,2025-11-06-17-53-57 +0000,nypost,"Zohran Mamdani ally Linda Sarsour, NYC DSA pol...",https://nypost.com/2025/11/06/us-news/zohran-m...
80,48,2025-11-06-21-16-00 +0000,wsj,The Bank of Mexico extended its monetary easin...,https://www.wsj.com/articles/bank-of-mexico-cu...
372,47,2025-11-05-23-28-53 +0000,nyt,Two Republican Incumbents Will Face Off as Red...,https://www.nytimes.com/2025/11/05/us/politics...
44,46,2025-11-06-22-09-16 +0000,nypost,Tesla shareholders approve Elon Musk’s histori...,https://nypost.com/2025/11/06/business/tesla-s...
28,45,2025-11-06-22-27-00 +0000,wsj,Trump Administration Must Fully Fund SNAP Bene...,https://www.wsj.com/politics/policy/trump-admi...
146,39,2025-11-06-18-39-43 +0000,nypost,CBS News’ Bari Weiss beefs up security detail ...,https://nypost.com/2025/11/06/media/cbs-news-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
